In [33]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict, load_model, Metrics, save_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
# process_train_data()
# process_test_data()

train_data, test_data = load_split_processed_data()

model_params = {
    "random_state": 42,
    "seed": 42,
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "feature_pre_filter": False,
    "lambda_l1": 2.740292203572801,
    "lambda_l2": 9.502614592851208,
    "learning_rate": 0.07586188568634152,
    "num_leaves": 93,
    "feature_fraction": 0.34049127666480095,
    "bagging_fraction": 0.8460348477289328,
    "max_depth": 10,
    "early_stopping_rounds": 88,
}


train_x = train_data.drop(columns="target")
train_y = train_data.target

valid_x = test_data.drop("target", axis=1)[train_x.columns]
valid_y = test_data.target

dtrain = lgb.Dataset(train_x, label=train_y)
dvalid = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)

model = lgb.train(
    model_params,
    dtrain,
    valid_sets=[dvalid],
)

print("Columns: ", train_x.columns)

X = test_data[train_x.columns]
y_true = test_data.target

y_pred_proba = model.predict(X, num_iteration=model.best_iteration)
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

Metrics().call(y_true, y_pred, y_pred_proba)

Columns:  Index(['imei_mean_days_usage', 'voice_mts_in_dwork_part_mea_mnt3',
       'data_3g_tar_vol_mea_diff', 'data_3g_tv_cnt_std_mnt3',
       'content_cnt_mea_diff', 'num_act_days_mea_mnt3', 'Balance_uah',
       'SUM_of_Duration_sec_susp_app_share', 'content_cnt_std_diff',
       'non_accum_internet_vol_std_diff',
       ...
       'accum_oth_dur_td_total', 'data_3g_tv_cnt_td_total',
       'voice_in_td_cnt_min_total', 'abon_part_td_total',
       'com_num_part_std_total', 'sms_in_cnt_std_total',
       'pay_p2p_out_sum_td_total', 'pay_max_td_total',
       'voice_in_short_part_td_total', 'voice_out_cmpttrs_avg_dumin_total'],
      dtype='object', length=382)

Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.72      0.32      0.44      9403

    accuracy                           0.95    150000
   macro avg       0.84      0.65      0.71    150000
weighted avg 

In [25]:
save_model(model, "LightGbmV2_new_features_fe_feature_selection_new_p072_r031_090auc", list(train_x.columns))

Save LightGbmV2_new_features_fe_feature_selection_new_p072_r031_090auc


'LightGbmV2_new_features_fe_feature_selection_new_p072_r031_090auc'